## Task description


Find the prices of a EU call, with S0 = K = 50, r = 0.1 (annual), σ = 0.4,
and maturity in five months using Monte Carlo methods, but use
antithetic variates, stratified sampling (use two strata, no need to use the
optimal weight), and control variates (use the underlying price as control)
as variance reduction techniques. Report the confidence intervals.



Use Quasi-Monte Carlo method to price the same EU call above (both
Halton and Weyl sequence). Report the confidence intervals.


In [2]:
# Import packages
import numpy as np
from scipy.stats import truncnorm, norm  # Truncated and normal continuous random variable
from scipy.stats import qmc

- Missing the Anthetic sample and Stratified sampling

## MC Method

In [8]:
# =====================================================
# Basic Monte Carlo Method
# =====================================================
def basic_monte_carlo(S0, K, r, sigma, T, N):
    # Setting standard normal random variables Z
    Z = np.random.randn(N)

    # Computing the expected price of the Stock
    S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    # Expected payoff, discounted
    payoff = np.exp(-r * T) * np.maximum(S_T - K, 0)

    # The european option price is the average of all the expected payoffs
    option_price = np.mean(payoff)

    # Computing Standard error for the confidence interval
    std_error = np.std(payoff, ddof=1) / np.sqrt(N)
    conf_interval = (option_price - 1.96 * std_error, option_price + 1.96 * std_error)
    return option_price, conf_interval


# =====================================================
# MC with Anthetic Variates
# =====================================================
def european_call_anthetic_variates(S0, K, r, sigma, T, N):
    M = N // 2   # Only half of the simulations
    Z = np.random.randn(M)
    Z_anthetic = -Z
    Z_combined = np.concatenate([Z, Z_anthetic])
    S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z_combined)
    payoff = np.maximum(S_T - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoff)
    std_error = np.exp(-r * T) * np.std(payoff, ddof=1) / np.sqrt(N)
    conf_interval = (option_price - 1.96 * std_error, option_price + 1.96 * std_error)
    return option_price, conf_interval


# =====================================================
# MC with Stratefied Sampling
# =====================================================
def european_call_stratified_sampling(S0, K, r, sigma, T, N):
    M = N // 2  # Half the simulations for each stratum
    # Stratum 1: U[0, 0.5]
    U1 = np.random.uniform(0, 0.5, M)
    Z1 = norm.ppf(U1)
    # Stratum 2: U[0.5, 1]
    U2 = np.random.uniform(0.5, 1, M)
    Z2 = norm.ppf(U2)
    Z = np.concatenate([Z1, Z2])
    S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    payoff = np.maximum(S_T - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoff)
    std_error = np.exp(-r * T) * np.std(payoff, ddof=1) / np.sqrt(N)
    conf_interval = (option_price - 1.96 * std_error, option_price + 1.96 * std_error)
    return option_price, conf_interval

# =====================================================
# MC with Control Variates
# =====================================================
def european_call_control_variates(S0, K, r, sigma, T, N):
    Z = np.random.randn(N)
    S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    payoff = np.maximum(S_T - K, 0)
    # Control variate: Use the expected price of the underlying asset
    E_S_T = S0 * np.exp(r * T)
    cov_payoff_S_T = np.cov(payoff, S_T, ddof=1)[0, 1]
    var_S_T = np.var(S_T, ddof=1)
    c = -cov_payoff_S_T / var_S_T  # Negative because we subtract the control variate
    adjusted_payoff = payoff + c * (S_T - E_S_T)
    option_price = np.exp(-r * T) * np.mean(adjusted_payoff)
    std_error = np.exp(-r * T) * np.std(adjusted_payoff, ddof=1) / np.sqrt(N)
    conf_interval = (option_price - 1.96 * std_error, option_price + 1.96 * std_error)
    return option_price, conf_interval


In [9]:
if __name__ == "__main__":
    # Parameters
    S0 = 50
    K = 50
    r = 0.1
    sigma = 0.4
    T = 5/12  # Five months
    N = 16000  # Number of simulations

    # Basic Monte Carlo
    basic_price, basic_ci = basic_monte_carlo(S0, K, r, sigma, T, N)
    print(f"Basic MC Option Price: {basic_price:.4f}")
    print(f"95% Confidence Interval: [{basic_ci[0]:.4f}, {basic_ci[1]:.4f}]\n")

    # Antithetic Variates
    av_price, av_ci = european_call_anthetic_variates(S0, K, r, sigma, T, N)
    print(f"Antithetic Variates Option Price: {av_price:.4f}")
    print(f"95% Confidence Interval: [{av_ci[0]:.4f}, {av_ci[1]:.4f}]\n")

    # Stratified Sampling
    ss_price, ss_ci = european_call_stratified_sampling(S0, K, r, sigma, T, N)
    print(f"Stratified Sampling Option Price: {ss_price:.4f}")
    print(f"95% Confidence Interval: [{ss_ci[0]:.4f}, {ss_ci[1]:.4f}]\n")

    # Control Variates
    cv_price, cv_ci = european_call_control_variates(S0, K, r, sigma, T, N)
    print(f"Control Variates Option Price: {cv_price:.4f}")
    print(f"95% Confidence Interval: [{cv_ci[0]:.4f}, {cv_ci[1]:.4f}]\n")

Basic MC Option Price: 6.1393
95% Confidence Interval: [5.9917, 6.2869]

Antithetic Variates Option Price: 6.0972
95% Confidence Interval: [5.9522, 6.2422]

Stratified Sampling Option Price: 6.1160
95% Confidence Interval: [5.9689, 6.2631]

Control Variates Option Price: 6.0714
95% Confidence Interval: [6.0149, 6.1279]



## Quasi-Monte Carlo (both Halton and Weyl sequence)

In [3]:
# We need to build the Weyl Sequences ourselves 

def generate_weyl_sequence(N, shift=0.5):
    """
    Generates a Weyl Sequence using a deterministic approach based on the golden ratio conjugate.
    Returns: A sequence of points (N) in [0, 1]
    """

    # Golden ratio conjugate
    alpha = (np.sqrt(5) - 1) / 2

    # Generate the Weyl sequence
    sequence = (np.arange(1, N, +1)* alpha + shift) % 1
    return sequence

In [4]:
# Defining the pricing for EU Call by using QMC by either Halton or Weyl
def price_european_call_qmc(S0, K, r, sigma, T, N, sequence_type = "halton"):

# ====================================================
# Sequence Type
# ====================================================
    if sequence_type.lower() == "halton":
        # Initialize Halton sampler
        sampler = qmc.Halton(d=1, scramble=True, seed=42)
        sample = sampler.random(N)
        U = sample.flatten()
    else:
        U = generate_weyl_sequence(N)

# ===============================================================
# Transforming to standard normal using inverse CDF
# ===============================================================
    Z = norm.ppf(U)

# Handle infinities resultin from norm.ppf(0) or norm.ppf(1)
    Z = np.where(Z == -np.inf, -10, Z)
    Z = np.where(Z == np.inf, 10, Z)

# ===============================================================
# Asset price at maturity using GBM, payoffs price and statistics
# ===============================================================
    S_T = S0 * np.exp((r - 0.5 * sigma**2)* T + sigma * np.sqrt(T) * Z)

# Payoff for EU call
    payoff = np.maximum(S_T - K, 0)

# Discount payoffs to present value
    discounted_payoff = np.exp(-r * T) * payoff

# Estimate option price as the mean of discounted payoffs
    option_price = np.mean(discounted_payoff)

# Standard Error using Sample
    std_error = np.std(discounted_payoff, ddof=1) / np.sqrt(N)

# Compute 95% CI
    conf_interval = (option_price- 1.96*std_error, option_price + 1.96*std_error)

    return option_price, conf_interval

def main():
    # Option Parameters
    S0 = 50            # Initial stock price
    K = 50             # Strike price
    r = 0.1            # Risk-free interest rate (annual)
    sigma = 0.4        # Volatility (annual)
    T = 5/12           # Time to maturity in years (5 months)
    N = 16000          # Number of simulations
    
    # Price using Halton sequence
    halton_price, halton_conf = price_european_call_qmc(
        S0, K, r, sigma, T, N, sequence_type='halton'
    )
    
    # Price using Weyl sequence
    weyl_price, weyl_conf = price_european_call_qmc(
        S0, K, r, sigma, T, N, sequence_type='weyl'
    )
     # Output Results
    print("European Call Option Pricing using Quasi-Monte Carlo Methods")
    print("-----------------------------------------------------------")
    print(f"Number of Simulations: {N}\n")
    
    print("1. Halton Sequence:")
    print(f"   Option Price Estimate: {halton_price:.4f}")
    print(f"   95% Confidence Interval: [{halton_conf[0]:.4f}, {halton_conf[1]:.4f}]\n")
    
    print("2. Weyl Sequence:")
    print(f"   Option Price Estimate: {weyl_price:.4f}")
    print(f"   95% Confidence Interval: [{weyl_conf[0]:.4f}, {weyl_conf[1]:.4f}]\n")


if __name__ == "__main__":
    main()

European Call Option Pricing using Quasi-Monte Carlo Methods
-----------------------------------------------------------
Number of Simulations: 16000

1. Halton Sequence:
   Option Price Estimate: 6.1165
   95% Confidence Interval: [5.9701, 6.2629]

2. Weyl Sequence:
   Option Price Estimate: 6.1172
   95% Confidence Interval: [5.9708, 6.2636]

